## 1. Coleta de Dados

1.1. Criação da conexão

1.2. Consulta na tabela

1.3. Tratar os dados para o receber o modelo treinado do arquivo 02.

## Objetivo do Projeto

Este processo tem como objetivo realizar a conexão com o banco de dados e extrair as principais informações que servirão de base para alertar o cliente sobre diferentes aspectos, como consumo de energia acima do normal, necessidade de manutenção em equipamentos, e o valor projetado da próxima conta de energia com base no consumo atual (diário), entre outros insights relevantes.

In [16]:
import pandas as pd
import oracledb
import warnings
warnings.filterwarnings("ignore")
import random
from datetime import datetime, timedelta


**Autenticação**

In [17]:
credencial_banco = {
    'dsn': 'oracle.fiap.com.br:1521/orcl',
    'usuario': 'rm553472',
    'senha': '100593'
}

**Conexão**

In [18]:
def conectar(usuario, senha, dsn):
    try:
        conexao = oracledb.connect(user=usuario, password=senha, dsn=dsn, mode=oracledb.DEFAULT_AUTH)
        print("Conexão com o banco de dados Oracle estabelecida com sucesso.")
        return conexao
    except oracledb.DatabaseError as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

conexao = conectar(usuario='rm553472',senha='100593', dsn='oracle.fiap.com.br:1521/orcl')

query_usuario = """
select * from t_usuario
"""

df_usuario = pd.read_sql(query_usuario, conexao)
df_usuario

Conexão com o banco de dados Oracle estabelecida com sucesso.


,ID_USUARIO,NOME,SOBRENOME,TELEFONE,EMAIL,SENHA,ID_ENDERECO
0,1,João,Silva,11987654321,joao.silva@email.com,senha123,1
1,2,Maria,Oliveira,11976543210,maria.oliveira@email.com,senha456,2
2,3,Carlos,Almeida,11987654322,carlos.almeida@email.com,senha789,3
3,4,Ana,Costa,11987654323,ana.costa@email.com,senha101,4
4,5,Pedro,Souza,11987654324,pedro.souza@email.com,senha202,5
5,6,Lucia,Mendes,11987654325,lucia.mendes@email.com,senha303,6
6,7,Ricardo,Lima,11987654326,ricardo.lima@email.com,senha404,7
7,8,Patricia,Martins,11987654327,patricia.martins@email.com,senha505,8
8,9,Juliana,Pereira,11987654328,juliana.pereira@email.com,senha606,9
9,10,Fernando,Silveira,11987654329,fernando.silveira@email.com,senha707,10


**Receber dados da tabela comodo**

In [19]:
query_comodo = """
select * from t_comodo
"""

df_comodo = pd.read_sql(query_comodo, conexao)
df_comodo

,ID_COMODO,ID_USUARIO,DESCRICAO
0,1,1,Sala
1,2,1,Quarto
2,3,1,Cozinha
3,4,1,Banheiro
4,5,1,Escritório
5,6,1,Garagem
6,7,1,Sala de Estar
7,8,1,Varanda
8,9,1,Lavanderia
9,10,1,Closet


**Receber dados da tabela itens dos comodos**

In [20]:
query_itens = """
select * from t_item_casa
"""

df_itens = pd.read_sql(query_itens, conexao)
df_itens

,ID_ITEM_CASA,ID_COMODO,DESCRICAO
0,1,1,Televisão
1,2,1,Som
2,3,1,Soundbar
3,4,2,Computador
4,5,2,Laptop
5,6,2,Ar-condicionado
6,7,3,Microondas
7,8,3,Geladeira
8,9,3,Forno
9,10,3,Batedeira


**Receber dados da tabela consumo**

In [21]:
query_consumo = """
select * from t_consumo
"""

df_consumo = pd.read_sql(query_consumo, conexao)
df_consumo

#df_consumo.to_csv('df_consumo.csv', index=False)

,ID_CONSUMO,ID_USUARIO,ID_COMODO,ID_ITEM_CASA,CONSUMO,DATA_CONSUMO
0,1,1,1,1,5.0,2024-11-01
1,2,2,2,2,3.0,2024-11-02
2,3,3,3,3,4.0,2024-11-03
3,4,4,4,4,2.0,2024-11-04
4,5,5,5,5,7.0,2024-11-05
5,6,6,6,6,1.0,2024-11-06
6,7,7,7,7,3.0,2024-11-07
7,8,8,8,8,6.0,2024-11-08
8,9,9,9,9,2.0,2024-11-09
9,10,10,10,10,4.0,2024-11-10


**Combinar as bases de dados**

In [22]:
# Tabela usuário + ambientes/comodos

df_usuario_comodo = pd.merge(df_usuario, df_comodo, on='ID_USUARIO', how='left')
df_usuario_comodo

# Filtrar a tabela só para mostrar o ID_USUARIO = 1
df_usuario_comodo = df_usuario_comodo[df_usuario_comodo['ID_USUARIO'] == 1]

# Transformar a coluna df_usuario_comodo em inteiro
df_usuario_comodo['ID_COMODO'] = df_usuario_comodo['ID_COMODO'].astype(int)
df_usuario_comodo

,ID_USUARIO,NOME,SOBRENOME,TELEFONE,EMAIL,SENHA,ID_ENDERECO,ID_COMODO,DESCRICAO
0,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,1,Sala
1,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,2,Quarto
2,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,3,Cozinha
3,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,4,Banheiro
4,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,5,Escritório
5,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,6,Garagem
6,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,7,Sala de Estar
7,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,8,Varanda
8,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,9,Lavanderia
9,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,10,Closet


In [23]:
# Tabela usuario_comodo + itens
df_usuario_comodo_itens = pd.merge(df_usuario_comodo, df_itens, on='ID_COMODO', how='left')

# Preencher a coluna df_usuario_comodo_itens['ID_ITEM_CASA'] com 0
df_usuario_comodo_itens['ID_ITEM_CASA'] = df_usuario_comodo_itens['ID_ITEM_CASA'].fillna(0)

# Transformar a coluna ID_ITEM_CASA em inteiro
df_usuario_comodo_itens['ID_ITEM_CASA'] = df_usuario_comodo_itens['ID_ITEM_CASA'].astype(int)
df_usuario_comodo_itens

,ID_USUARIO,NOME,SOBRENOME,TELEFONE,EMAIL,SENHA,ID_ENDERECO,ID_COMODO,DESCRICAO_x,ID_ITEM_CASA,DESCRICAO_y
0,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,1,Sala,1,Televisão
1,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,1,Sala,2,Som
2,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,1,Sala,3,Soundbar
3,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,2,Quarto,4,Computador
4,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,2,Quarto,5,Laptop
5,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,2,Quarto,6,Ar-condicionado
6,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,3,Cozinha,7,Microondas
7,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,3,Cozinha,8,Geladeira
8,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,3,Cozinha,9,Forno
9,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,3,Cozinha,10,Batedeira


In [24]:
# Tabela usuario_comodo_itens + consumo
df_final_combinado = pd.merge(df_usuario_comodo_itens, df_consumo, on=['ID_USUARIO', 'ID_COMODO', 'ID_ITEM_CASA'], how='left')
df_final_combinado.head()

,ID_USUARIO,NOME,SOBRENOME,TELEFONE,EMAIL,SENHA,ID_ENDERECO,ID_COMODO,DESCRICAO_x,ID_ITEM_CASA,DESCRICAO_y,ID_CONSUMO,CONSUMO,DATA_CONSUMO
0,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,1,Sala,1,Televisão,1.0,5.0,2024-11-01
1,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,1,Sala,2,Som,NaN,NaN,NaT
2,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,1,Sala,3,Soundbar,NaN,NaN,NaT
3,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,2,Quarto,4,Computador,NaN,NaN,NaT
4,1,João,Silva,11987654321,joao.silva@email.com,senha123,1,2,Quarto,5,Laptop,NaN,NaN,NaT


**Salvar os dados tratados e usar no teste final do arquivo 02**

In [25]:
df_final_combinado = df_final_combinado[['DATA_CONSUMO','DESCRICAO_x','DESCRICAO_y','CONSUMO']]

# Renomear as colunas do dataframe df_final_combinado
df_final_combinado = df_final_combinado.rename(columns={'DATA_CONSUMO': 'data','DESCRICAO_x':'ambiente', 'DESCRICAO_y':'item', 'CONSUMO':'consumo_kwh'})

# Inserir uma coluna turno com valor 'Tarde'
df_final_combinado['turno'] = 'Tarde'

# Ajustar ordem das colunas
df_final_combinado = df_final_combinado[['data','turno','ambiente','item','consumo_kwh']]

df_final_combinado

#df_consumo.to_csv('df_consumo.csv', index=False)

,data,turno,ambiente,item,consumo_kwh
0,2024-11-01,Tarde,Sala,Televisão,5.0
1,NaT,Tarde,Sala,Som,NaN
2,NaT,Tarde,Sala,Soundbar,NaN
3,NaT,Tarde,Quarto,Computador,NaN
4,NaT,Tarde,Quarto,Laptop,NaN
5,NaT,Tarde,Quarto,Ar-condicionado,NaN
6,NaT,Tarde,Cozinha,Microondas,NaN
7,NaT,Tarde,Cozinha,Geladeira,NaN
8,NaT,Tarde,Cozinha,Forno,NaN
9,NaT,Tarde,Cozinha,Batedeira,NaN


**Gerar uma base para nos ajudar a montar nosso projeto e mostrar valor projetado**

In [26]:

# Dados de ambientes e itens
ambientes = ['sala', 'cozinha', 'quarto1', 'quarto2', 'banheiro', 'lavanderia', 'sacada']
itens = {
    'sala': ['ar-condicionado', 'TV', 'luz'],
    'cozinha': ['geladeira', 'micro-ondas', 'luz'],
    'quarto1': ['ar-condicionado', 'luz'],
    'quarto2': ['computador', 'luz'],
    'banheiro': ['aquecedor', 'luz'],
    'lavanderia': ['máquina de lavar'],
    'sacada': ['luz']
}

##Turnos:
##Turno 1 (Manhã): das 06:00 às 12:00
##Turno 2 (Tarde): das 12:00 às 18:00
##Turno 3 (Noite): das 18:00 às 00:00
##Turno 4 (Madrugada): das 00:00 às 06:00

# Função para gerar o consumo diário
def gerar_consumo_diario():
    # Consumo entre 0.5 kWh e 5 kWh
    return round(random.uniform(0.5, 5.0), 2)

# Função para calcular o custo baseado em tarifas diferenciadas
def calcular_custo(consumo_kwh, turno):
    # Tarifas (podem ser ajustadas conforme os turnos)
    tarifas = {
        'manha': 0.60,  # Turno Manhã: R$ 0,60 por kWh
        'tarde': 0.60,  # Turno Tarde: R$ 0,60 por kWh
        'noite': 0.75,  # Turno Noite (Ponta): R$ 0,75 por kWh
        'madrugada': 0.50  # Turno Madrugada: R$ 0,50 por kWh
    }
    
    # Aplique a tarifa do turno
    tarifa = tarifas.get(turno, 0.60)  # Se o turno não for encontrado, assume tarifa de manhã/tarde
    
    # Impostos e taxas (porcentagens)
    icms = 0.18  # ICMS (18%)
    pis_cofins = 0.0384  # PIS/COFINS (3,84%)
    outros_encargos = 0.05  # Outros encargos (5%)
    
    # Calculando o custo
    custo_base = consumo_kwh * tarifa
    custo_impostos = custo_base * (icms + pis_cofins)
    custo_encargos = custo_base * outros_encargos
    
    # Cálculo final com impostos e encargos
    custo_total = custo_base + custo_impostos + custo_encargos
    return round(custo_total, 2)

# Função para atribuir o turno baseado na data (sem considerar hora exata)
def determinar_turno(data):
    # Atribuir turnos com base no dia da semana e sua posição no mês
    if data.weekday() < 5:  # Segunda a Sexta (turnos regulares)
        if data.day % 4 == 0:
            return 'noite'  # Simular que o consumo maior é à noite (pico)
        else:
            return 'manha'  # Fora ponta, simula o consumo durante o dia
    else:  # Fim de semana, o consumo pode ser aleatório
        return random.choice(['tarde', 'madrugada'])  # Simula consumo mais fora do horário de pico

# Gerar dados para 2000 linhas
dados = []
data_inicio = datetime(2023, 1, 1)
for i in range(2000):
    data_consumo = data_inicio + timedelta(days=i)
    ambiente = random.choice(ambientes)
    item = random.choice(itens[ambiente])
    consumo_diario = gerar_consumo_diario()
    
    # Determinar o turno
    turno = determinar_turno(data_consumo)
    
    # Calcular o custo com base no turno
    custo = calcular_custo(consumo_diario, turno)
    
    # Adicionar linha de dados
    dados.append({
        'data': data_consumo.strftime('%Y-%m-%d'),
        'turno': turno,
        'ambiente': ambiente,
        'item': item,
        'consumo_kwh': consumo_diario,
        'custo_estimado': custo
    })

# Criar o DataFrame
df = pd.DataFrame(dados)

# Exibir as primeiras linhas do DataFrame
df.head(10)

# Ajustar colunas e salvar um modelo para teste
df = df[['data','turno','ambiente','item','consumo_kwh']]

# Salvar os dados em um arquivo CSV
df.to_csv('base_cliente', index=False)


**Fechar a conexão com o banco**

In [27]:
conexao.close()

## Informações adicionais

Todo o processo descrito acima fornecerá insumos para gerar projeções dos valores que o cliente poderá acompanhar por meio do site ou do aplicativo móvel. Assim, o cliente não precisará esperar a chegada da conta de energia para saber o quanto consumiu e quanto terá que desembolsar no próximo pagamento.

O projeto também busca conscientizar os usuários sobre a redução de gastos, identificando os equipamentos e horários de maior consumo. Além disso, poderá antecipar a necessidade de manutenções em equipamentos que apresentem sinais de problemas, como consumo muito abaixo do normal (no caso de geladeiras), indicando que o aparelho já não está funcionando conforme suas especificações. Também será possível identificar casos como lâmpadas que consomem mais energia do que o normal, permitindo ao cliente trocá-las por opções mais eficientes, como lâmpadas de LED, entre outras melhorias.